# Sitzung 2 – RAG-Bewertung mit ragas

Bewertung einer minimalen RAG-Pipeline mithilfe von ragas-Metriken: Antwortrelevanz, Zuverlässigkeit, Kontextpräzision.


# Szenario
Dieses Szenario bewertet eine minimale Retrieval Augmented Generation (RAG)-Pipeline lokal. Wir:
- Definieren ein kleines synthetisches Dokumenten-Corpus.
- Betten Dokumente ein und implementieren einen einfachen Ähnlichkeits-Retriever.
- Generieren fundierte Antworten mithilfe eines lokalen Modells (Foundry Local / OpenAI-kompatibel).
- Berechnen ragas-Metriken (`answer_relevancy`, `faithfulness`, `context_precision`).
- Unterstützen einen SCHNELL-Modus (Umgebung `RAG_FAST=1`), um nur die Antwortrelevanz für schnelle Iterationen zu berechnen.

Verwenden Sie dieses Notebook, um zu validieren, dass Ihr lokales Modell + Embedding-Stack faktisch fundierte Antworten liefert, bevor Sie auf größere Korpora skalieren.


### Erklärung: Abhängigkeitsinstallation
Installiert die benötigten Bibliotheken:
- `foundry-local-sdk` für die Verwaltung lokaler Modelle.
- `openai` für die Client-Schnittstelle.
- `sentence-transformers` für dichte Einbettungen.
- `ragas` + `datasets` für Evaluierung und Metrikberechnung.
- `langchain-openai` Adapter für die Ragas-LLM-Schnittstelle.

Kann bedenkenlos erneut ausgeführt werden; überspringen, wenn die Umgebung bereits vorbereitet ist.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Erklärung: Kernimporte & Metriken
Lädt Kernbibliotheken und Ragas-Metriken. Wichtige Bestandteile:
- SentenceTransformer für Embeddings.
- `evaluate` + ausgewählte Ragas-Metriken.
- `Dataset` zum Erstellen des Evaluierungskorpus.
Diese Importe lösen keine Remote-Aufrufe aus (außer möglicherweise das Laden des Modell-Caches für Embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Erklärung: Spielzeugkorpus & QA-Ground-Truth
Definiert einen kleinen In-Memory-Korpus (`DOCS`), eine Reihe von Benutzerfragen und die erwarteten Ground-Truth-Antworten. Diese ermöglichen eine schnelle, deterministische Metrikberechnung ohne externe Datenabfragen. In realen Szenarien würden Sie Produktionsanfragen und kuratierte Antworten verwenden.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Erklärung: Service-Initialisierung, Embeddings & Sicherheits-Patch
Initialisiert den Foundry Local Manager, wendet einen Sicherheits-Patch für Schema-Abweichungen bei `promptTemplate` an, löst die Modell-ID auf, erstellt einen OpenAI-kompatiblen Client und berechnet vorab dichte Embeddings für den Dokumentenkorpus. Dies richtet einen wiederverwendbaren Zustand für Abruf + Generierung ein.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Erklärung: Retriever-Funktion
Definiert einen einfachen Vektor-Similaritäts-Retriever, der den Skalarprodukt über normalisierte Einbettungen verwendet. Gibt die Top-k-Dokumente zurück (Standard k=2). In der Produktion sollte dies durch einen ANN-Index (FAISS, Chroma, Milvus) ersetzt werden, um Skalierbarkeit und geringe Latenz zu gewährleisten.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Erklärung: Generierungsfunktion
`generate` erstellt eine eingeschränkte Eingabeaufforderung (das System weist an, NUR den Kontext zu verwenden) und ruft das lokale Modell auf. Eine niedrige Temperatur (0,1) begünstigt eine genaue Extraktion gegenüber Kreativität. Gibt den gekürzten Antworttext zurück.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Erklärung: Fallback-Initialisierung des Clients
Stellt sicher, dass `client` existiert, selbst wenn eine frühere Initialisierungszelle übersprungen oder fehlgeschlagen ist – verhindert einen NameError bei späteren Auswertungsschritten.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Erklärung: Evaluationsschleife & Metriken
Erstellt den Evaluierungsdatensatz (erforderliche Spalten: Frage, Antwort, Kontexte, Ground Truths, Referenz) und durchläuft anschließend die ausgewählten Ragas-Metriken.

Optimierung:
- FAST_MODE beschränkt sich auf die Relevanz der Antwort für schnelle Smoke-Tests.
- Pro-Metrik-Schleife vermeidet eine vollständige Neuberechnung, wenn eine Metrik fehlschlägt.

Gibt ein Dictionary von Metrik -> Score aus (NaN bei Fehler).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
